In [1]:
# librerie

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql import Row
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.feature import StandardScaler
from pyspark.mllib.clustering import KMeans
from pyspark.ml.clustering import BisectingKMeans

In [2]:
# sessione

sc = SparkContext(appName="DDAM_Project", master="local[*]")
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("DDAM_Project") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
sdf = spark.read.csv("hdfs://kddrtserver11.isti.cnr.it:9000/user/hpsa04/credit_train.csv", sep=",",
                     inferSchema=True, header=True)

columns = sdf.schema.names

# rinominare le colonne sotituendo lo spazio con l'underscore
for col in columns:
    sdf = sdf.withColumnRenamed(col, col.replace(' ', '_'))

columns = sdf.schema.names

sdf.printSchema()

root
 |-- Loan_ID: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Loan_Status: string (nullable = true)
 |-- Current_Loan_Amount: integer (nullable = true)
 |-- Term: string (nullable = true)
 |-- Credit_Score: integer (nullable = true)
 |-- Annual_Income: integer (nullable = true)
 |-- Years_in_current_job: string (nullable = true)
 |-- Home_Ownership: string (nullable = true)
 |-- Purpose: string (nullable = true)
 |-- Monthly_Debt: double (nullable = true)
 |-- Years_of_Credit_History: double (nullable = true)
 |-- Months_since_last_delinquent: string (nullable = true)
 |-- Number_of_Open_Accounts: integer (nullable = true)
 |-- Number_of_Credit_Problems: integer (nullable = true)
 |-- Current_Credit_Balance: integer (nullable = true)
 |-- Maximum_Open_Credit: integer (nullable = true)
 |-- Bankruptcies: string (nullable = true)
 |-- Tax_Liens: string (nullable = true)



### - Funzioni

In [4]:
def get_nbr_nulls(spark_df, view_name, print_result = True):
    """funzione per ottenere il numero di valori nulli presenti in ogni attributo"""
    
    spark_df.createOrReplaceTempView(view_name)
    
    columns_temp = spark_df.schema.names
    
    Project = []
    for col in columns_temp:
        Project.append('SUM(CASE WHEN {0} IS NULL THEN 1 ELSE 0 END) AS {0}'.format(col))
    Project = ', '.join(Project)

    sql = """\
    SELECT {0}
    FROM {1}\
    """.format(Project, view_name)
    
    nbr_nulls = spark.sql(sql).first().asDict()
    
    if print_result:
        for key, value in nbr_nulls.items():
            print(key + ':', '{:>10}'.format(value))
        
    return nbr_nulls

In [5]:
nbr_nulls = get_nbr_nulls(spark_df = sdf, view_name = 'Bank_Loan_Dataset')

Loan_ID:        514
Customer_ID:        514
Loan_Status:        514
Current_Loan_Amount:        514
Term:        514
Credit_Score:      19668
Annual_Income:      19668
Years_in_current_job:        514
Home_Ownership:        514
Purpose:        514
Monthly_Debt:        514
Years_of_Credit_History:        514
Months_since_last_delinquent:        514
Number_of_Open_Accounts:        514
Number_of_Credit_Problems:        514
Current_Credit_Balance:        514
Maximum_Open_Credit:        516
Bankruptcies:        514
Tax_Liens:        514


In [6]:
def get_nbr_distincts(spark_df, view_name, print_result = True):
    """funzione per ottenere il numero di valori distinti di ciascun attributo.
    il valore nullo non viene contato come valore distinto"""
    
    spark_df.createOrReplaceTempView(view_name)
    
    columns_temp = spark_df.schema.names

    Project = []
    for col in columns_temp:
        Project.append('COUNT(DISTINCT {0}) AS {0}'.format(col))
    Project = ', '.join(Project)

    sql = """\
    SELECT {0}
    FROM {1}\
    """.format(Project, view_name)
    
    nbr_distincts = spark.sql(sql).first().asDict()
    
    if print_result:
        for key, value in nbr_distincts.items():
            print(key + ':', '{:>10}'.format(value))

    return nbr_distincts

In [7]:
nbr_distincts = get_nbr_distincts(spark_df = sdf, view_name = 'Bank_Loan_Dataset')

Loan_ID:      81999
Customer_ID:      81999
Loan_Status:          2
Current_Loan_Amount:      22004
Term:          2
Credit_Score:        324
Annual_Income:      36174
Years_in_current_job:         12
Home_Ownership:          4
Purpose:         16
Monthly_Debt:      65765
Years_of_Credit_History:        506
Months_since_last_delinquent:        117
Number_of_Open_Accounts:         51
Number_of_Credit_Problems:         14
Current_Credit_Balance:      32730
Maximum_Open_Credit:      44596
Bankruptcies:          9
Tax_Liens:         13


# modifiche agli attributi

In [8]:
rdd = sdf.rdd  # di default questa trasformazione genera un RDD di Row()

'''le Row sono tipi particolari di Tuple, quindi sono oggetti immutabili.
in tutto il notebook allora per modificare gli RDD li trasformiamo temporaneamente (dentro le funzioni)
in RDD di Dictionaries.'''

'le Row sono tipi particolari di Tuple, quindi sono oggetti immutabili.\nin tutto il notebook allora per modificare gli RDD li trasformiamo temporaneamente (dentro le funzioni)\nin RDD di Dictionaries.'

gli attributi che riguardano somme di denaro sono quantità denominate in valuta russa (Rubli). Il dataset si riferisce a dati del 2016, quindi per rendere più comprensibile il significato di queste somme tutti questi attributi vengono convertiti in Euro dividendo tutti i loro valori per il tasso di cambio EUR/RUB medio arrotondato alle decine dell'anno 2016: 70.

questa modifica non impatta assolutamente nessuna analisi perché tutte le quantità monetarie vengono trasformate alla stessa maniera e quindi le proporzioni vengono mantenute. Sarà sempre possibile trasformare facilmente di nuovo in Rubli qualora sia necessario.

In [9]:
def change_value(row):
    d = row.asDict()
    
    if d['Current_Loan_Amount'] is not None:
        d['Current_Loan_Amount'] = int(d['Current_Loan_Amount']/70)
    
    if d['Annual_Income'] is not None:    
        d['Annual_Income'] = int(d['Annual_Income']/70)
    
    if d['Monthly_Debt'] is not None:
        d['Monthly_Debt'] = d['Monthly_Debt']/70
        
    if d['Current_Credit_Balance'] is not None:
        d['Current_Credit_Balance'] = int(d['Current_Credit_Balance']/70)
        
    if d['Maximum_Open_Credit'] is not None:
        d['Maximum_Open_Credit'] = int(d['Maximum_Open_Credit']/70)
    
    return Row(**d)

rdd = rdd.map(change_value)

#### ci sono alcuni attributi che hanno un Data Type incoerente con il significato dell'attributo: sono letti come stringhe ma in realtà la loro semantica ci suggerisce di trasformarli in numerici.

In [10]:
# Check dei valori distinti degli attributi con data type incoerente

problematic_columns = ['Years_in_current_job', 'Months_since_last_delinquent', 'Bankruptcies', 'Tax_Liens']

for col in problematic_columns:
    sdf.select(col).groupBy(col).count().orderBy('count', ascending=False).show(150)

+--------------------+-----+
|Years_in_current_job|count|
+--------------------+-----+
|           10+ years|31121|
|             2 years| 9134|
|             3 years| 8169|
|            < 1 year| 8164|
|             5 years| 6787|
|              1 year| 6460|
|             4 years| 6143|
|             6 years| 5686|
|             7 years| 5577|
|             8 years| 4582|
|                 n/a| 4222|
|             9 years| 3955|
|                null|  514|
+--------------------+-----+

+----------------------------+-----+
|Months_since_last_delinquent|count|
+----------------------------+-----+
|                          NA|53141|
|                          13|  922|
|                          12|  902|
|                          14|  877|
|                          15|  865|
|                          10|  861|
|                           8|  856|
|                           9|  849|
|                          18|  847|
|                          16|  837|
|                        

- 'Years_in_current_job'

rendiamo numerico quest'attributo in questo modo:

years viene tolto da ogni valore.

sono presenti 4222 valori uguali a 'n/a'. Pensiamo si possa trattare di soggetti per cui non si può dire qunati anni hanno lavorato nel lavoro corrente perché sono attualmente senza occupazione. Il valore viene quindi sostituito con 0.

10+ viene trasformato in 10 perché 10+ non ha valenza semantica non conoscendo la distribuzione dei valori specifici per questa categoria. Questa trasformazione non comporta problematiche per algoritmi di machine learning come il Decision Tree perché l'ordinamento dei valori numerici è preservato (basterà tenere presente che un eventuale split sul valore 10 sarebbe in realtà riferito a valori anche maggiori di 10). L'unica problematica apparente potrebbe manifestarsi per algoritmi basati sulle distanze (es. Clustering, PCA, KNN ecc...) perché la distanza dal valore 10 potrebbe in realtà essere una distanza molto maggiore. Ma non ci preoccupiamo di questo perché le ennuple coinvolte sono relativamente poche e l'errore non avrebbe un impatto rilevante sul calcolo della distanza multidimensionale. E anche soprattutto perché arrivati a 10 anni di lavoro in una posizione si ritiene che il fattore lavorativo non sia ormai più un problema e quindi un'ipotetica "distanza" ad esempio tra 10 e 25 anni è tutto sommato meno rilevante di una distanza, anche minore, ma ad esempio tra 10 e 1.

< 1 viene trasformato nella media dei valori tra 0 e 1, cioè 0.5.

In [11]:
def change_value(row):
    d = row.asDict()
    
    if d['Years_in_current_job'] is not None:
        d['Years_in_current_job'] = str(d['Years_in_current_job']).replace(' years', '')
        d['Years_in_current_job'] = str(d['Years_in_current_job']).replace(' year', '')
        
        if d['Years_in_current_job'] == 'n/a':
            d['Years_in_current_job'] = 0
        if d['Years_in_current_job'] == '10+':
            d['Years_in_current_job'] = 10
        if d['Years_in_current_job'] == '< 1':
            d['Years_in_current_job'] = 0.5
        
        d['Years_in_current_job'] = float(d['Years_in_current_job'])
    
    return Row(**d)

rdd = rdd.map(change_value)

In [12]:
'''def change_value(row):
    d = row.asDict()
    if d['Years_in_current_job'] is not None:
        
        if d['Years_in_current_job'] == '10+ years':
            d['Years_in_current_job'] = '>= 10 years'

        if (d['Years_in_current_job'] == '4 years') or (d['Years_in_current_job'] == '5 years') or (d['Years_in_current_job'] == '6 years') or (d['Years_in_current_job'] == '7 years') or (d['Years_in_current_job'] == '8 years') or (d['Years_in_current_job'] == '9 years'):
            d['Years_in_current_job'] = '>= 4 years and < 10 years'
            
        if (d['Years_in_current_job'] == '< 1 year') or (d['Years_in_current_job'] == '1 year') or (d['Years_in_current_job'] == '2 years') or (d['Years_in_current_job'] == '3 years'):
            d['Years_in_current_job'] = '< 4 years'
            
    return Row(**d)

rdd = rdd.map(change_value)'''

"def change_value(row):\n    d = row.asDict()\n    if d['Years_in_current_job'] is not None:\n        \n        if d['Years_in_current_job'] == '10+ years':\n            d['Years_in_current_job'] = '>= 10 years'\n\n        if (d['Years_in_current_job'] == '4 years') or (d['Years_in_current_job'] == '5 years') or (d['Years_in_current_job'] == '6 years') or (d['Years_in_current_job'] == '7 years') or (d['Years_in_current_job'] == '8 years') or (d['Years_in_current_job'] == '9 years'):\n            d['Years_in_current_job'] = '>= 4 years and < 10 years'\n            \n        if (d['Years_in_current_job'] == '< 1 year') or (d['Years_in_current_job'] == '1 year') or (d['Years_in_current_job'] == '2 years') or (d['Years_in_current_job'] == '3 years'):\n            d['Years_in_current_job'] = '< 4 years'\n            \n    return Row(**d)\n\nrdd = rdd.map(change_value)"

- 'Months_since_last_delinquent'

sono presenti 53141 valori uguali a 'NA' (la maggior parte). la nostra interpretazione di questo valore è che il soggetto in questione non ha mai commesso nessun reato (interpretazione coerente con il fatto che i valori 'NA' sono la maggior parte).

il conteggio degli altri valori distinti è basso, decidiamo quindi di rendere l'attributo categorico raccogliendo i valori in questi range:

[0, 12); [12, 48); [48, 96); [96, +inf); 'Never committed'


In [13]:
def change_value(row):
    d = row.asDict()
    
    if d['Months_since_last_delinquent'] is not None:
        if d['Months_since_last_delinquent'] == 'NA':
            d['Months_since_last_delinquent'] = 'Never committed'
        elif int(d['Months_since_last_delinquent']) < 12:
            d['Months_since_last_delinquent'] = '[0, 12)'
        elif int(d['Months_since_last_delinquent']) < 48:
            d['Months_since_last_delinquent'] = '[12, 48)'
        elif int(d['Months_since_last_delinquent']) < 96:
            d['Months_since_last_delinquent'] = '[48, 96)'
        else:
            d['Months_since_last_delinquent'] = '[96, +inf)'
            
    return Row(**d)

rdd = rdd.map(change_value)

In [14]:
'''def change_value(row):
    d = row.asDict()
    
    if d['Months_since_last_delinquent'] is None:
        d['Has_Been_Delinquent'] = None
    elif d['Months_since_last_delinquent'] == 'NA':
        d['Has_Been_Delinquent'] = '0'
    else:
        d['Has_Been_Delinquent'] = '1'
        
    del d['Months_since_last_delinquent']
    return Row(**d)

rdd = rdd.map(change_value)'''

"def change_value(row):\n    d = row.asDict()\n    \n    if d['Months_since_last_delinquent'] is None:\n        d['Has_Been_Delinquent'] = None\n    elif d['Months_since_last_delinquent'] == 'NA':\n        d['Has_Been_Delinquent'] = '0'\n    else:\n        d['Has_Been_Delinquent'] = '1'\n        \n    del d['Months_since_last_delinquent']\n    return Row(**d)\n\nrdd = rdd.map(change_value)"

- 'Bankruptcies'

l'attributo viene letto come stringa perché sono presenti 204 valori uguali a 'NA'. Poichè il valore 0 è presente si pensa possa trattarsi di missing values, li trasformiamo quindi in None e rendiamo numerico l'attributo.

In [15]:
def change_value(row):
    d = row.asDict()
    
    if d['Bankruptcies'] is not None:
        if d['Bankruptcies'] == 'NA':
            d['Bankruptcies'] = None
        else:
            d['Bankruptcies'] = int(d['Bankruptcies'])
        
    return Row(**d)

rdd = rdd.map(change_value)

- 'Tax_Liens'

l'attributo viene letto come stringa perché sono presenti 10 valori uguali a 'NA'. Poichè il valore 0 è presente si pensa possa trattarsi di missing values, li trasformiamo quindi in None e rendiamo numerico l'attributo.

In [16]:
def change_value(row):
    d = row.asDict()

    if d['Tax_Liens'] is not None:
        if d['Tax_Liens'] == 'NA':
            d['Tax_Liens'] = None
        else:
            d['Tax_Liens'] = int(d['Tax_Liens'])
        
    return Row(**d)

rdd = rdd.map(change_value)

In [17]:
sdf = rdd.toDF()
columns = sdf.schema.names
sdf.printSchema()

root
 |-- Annual_Income: long (nullable = true)
 |-- Bankruptcies: long (nullable = true)
 |-- Credit_Score: long (nullable = true)
 |-- Current_Credit_Balance: long (nullable = true)
 |-- Current_Loan_Amount: long (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Home_Ownership: string (nullable = true)
 |-- Loan_ID: string (nullable = true)
 |-- Loan_Status: string (nullable = true)
 |-- Maximum_Open_Credit: long (nullable = true)
 |-- Monthly_Debt: double (nullable = true)
 |-- Months_since_last_delinquent: string (nullable = true)
 |-- Number_of_Credit_Problems: long (nullable = true)
 |-- Number_of_Open_Accounts: long (nullable = true)
 |-- Purpose: string (nullable = true)
 |-- Tax_Liens: long (nullable = true)
 |-- Term: string (nullable = true)
 |-- Years_in_current_job: double (nullable = true)
 |-- Years_of_Credit_History: double (nullable = true)



In [18]:
for col in problematic_columns:
    sdf.select(col).groupBy(col).count().orderBy('count', ascending=False).show(150)

+--------------------+-----+
|Years_in_current_job|count|
+--------------------+-----+
|                10.0|31121|
|                 2.0| 9134|
|                 3.0| 8169|
|                 0.5| 8164|
|                 5.0| 6787|
|                 1.0| 6460|
|                 4.0| 6143|
|                 6.0| 5686|
|                 7.0| 5577|
|                 8.0| 4582|
|                 0.0| 4222|
|                 9.0| 3955|
|                null|  514|
+--------------------+-----+

+----------------------------+-----+
|Months_since_last_delinquent|count|
+----------------------------+-----+
|             Never committed|53141|
|                    [12, 48)|25789|
|                    [48, 96)|13453|
|                     [0, 12)| 7590|
|                        null|  514|
|                  [96, +inf)|   27|
+----------------------------+-----+

+------------+-----+
|Bankruptcies|count|
+------------+-----+
|           0|88774|
|           1|10475|
|        null|  718|
|        

# Gestione degli errori

alcuni attributi hanno valori errati o non coerenti con il significato che noi reputiamo possa avere l'attributo. Alcuni di questi errori sono stati scoperti in fasi più avanzate del progetto (ex Data Understanding).

- 'Credit_Score'

ci sono 4551 valori dell'attributo 'Credit_Score' con valore superiore a 5000, che è un valore troppo distante rispetto a quelli generici che assume questo attributo (da 500 a 800). Controllando i valori distinti si è scoperto che quei valori sono errati: c'è uno '0' di troppo in fondo al numero, che eliminamo.

In [19]:
def problematic_values_count(row):
    if row['Credit_Score'] is not None:
        return row['Credit_Score'] > 5000

rdd.filter(problematic_values_count).count()

4551

In [20]:
col = 'Credit_Score'
sdf.select(col).groupBy(col).count().orderBy('count', ascending=False).show(400)

+------------+-----+
|Credit_Score|count|
+------------+-----+
|        null|19668|
|         747| 1825|
|         740| 1746|
|         746| 1742|
|         741| 1732|
|         742| 1723|
|         739| 1624|
|         745| 1612|
|         748| 1598|
|         743| 1555|
|         725| 1548|
|         724| 1522|
|         738| 1495|
|         744| 1485|
|         721| 1465|
|         723| 1421|
|         737| 1405|
|         722| 1387|
|         718| 1261|
|         750| 1234|
|         717| 1218|
|         720| 1216|
|         736| 1156|
|         734| 1147|
|         735| 1134|
|         719| 1118|
|         732| 1084|
|         733| 1073|
|         715| 1070|
|         716| 1061|
|         714| 1046|
|         713| 1017|
|         731| 1010|
|         712| 1006|
|         730|  984|
|         728|  931|
|         729|  925|
|         708|  902|
|         709|  865|
|         710|  857|
|         726|  853|
|         749|  827|
|         707|  814|
|         727|  806|
|         711

In [21]:
def errors_correction(row):
    d = row.asDict()
    if d['Credit_Score'] is not None and d['Credit_Score'] > 5000:
        d['Credit_Score'] = int(str(d['Credit_Score'])[:-1])
    return Row(**d)
    
rdd = rdd.map(errors_correction)
rdd.count()

100514

- l'attributo 'Current_Credit_Balance' non può essere superiore al valore dell'attributo 'Maximum_Open_Credit'

# Current_Credit_Balance <= Current_Loan_Amount <= Maximum_Open_Credit

In [22]:
'''def errors_correction(row):
    if (row['Current_Credit_Balance'] is not None) and (row['Maximum_Open_Credit'] is not None):
        return row['Current_Credit_Balance'] <= row['Maximum_Open_Credit']
    else:
        return row
    
rdd = rdd.filter(errors_correction)
rdd.count()'''

"def errors_correction(row):\n    if (row['Current_Credit_Balance'] is not None) and (row['Maximum_Open_Credit'] is not None):\n        return row['Current_Credit_Balance'] <= row['Maximum_Open_Credit']\n    else:\n        return row\n    \nrdd = rdd.filter(errors_correction)\nrdd.count()"

- ci sono diversi valori dell'attributo 'Current_Loan_Amount' che sono pari a 1.428.571 (99.999.999 in Rubli).  un numero eccessivamente più alto rispetto a tutti gli altri valori e che per questo viene eliminato.

In [23]:
col = 'Current_Loan_Amount'
sdf.select(col).groupBy(col).count().orderBy('count', ascending=False).show()

+-------------------+-----+
|Current_Loan_Amount|count|
+-------------------+-----+
|            1428571|11484|
|               null|  514|
|               3195|   82|
|               3184|   70|
|               3088|   66|
|               3173|   66|
|               3140|   66|
|               1556|   65|
|               3179|   65|
|               1540|   63|
|               3069|   62|
|               3190|   60|
|               1919|   58|
|               3080|   58|
|               3107|   58|
|               1534|   58|
|               3157|   58|
|               3217|   57|
|               3078|   57|
|               3206|   56|
+-------------------+-----+
only showing top 20 rows



In [24]:
def errors_correction(row):
    if row['Current_Loan_Amount'] is not None:
        return row['Current_Loan_Amount'] != 1428571
    else:
        return row
    
rdd = rdd.filter(errors_correction)
rdd.count()

89030

# mximum open credit di 100 000 000

eliminiamo le righe dove entrambi gli attributi 'Loan_ID' e 'Customer_ID' sono nulli, che corrispondono alle righe dove tutti i valori di tutti gli attributi sono nulli

In [25]:
rdd = rdd.filter(lambda row: not ( (row['Loan_ID'] is None) and (row['Customer_ID'] is None) ))
rdd.count()

88516

eliminiamo le righe duplicate

In [26]:
rdd = rdd.distinct()
rdd.count()

78301

Problema: ci sono coppie di righe con tutti i valori duplicati eccetto per le due colonne 'Credit_Score' e 'Annual_Income', per le quali uno dei due valori è presente e l'altro è nullo.

Soluzione: si raggruppa per tutti gli attributi tranne quei due e poi si calcola la media di quei due. In questo modo se le uniche due righe uguali sono quelle con un valore nullo e uno non nullo per quegli attributi, lo media sarà uguale al valore non nullo; se invece ci fossero altre righe ugauli ma con altri valori diversi non nulli per quegli attributi, viene effettivamente calcolata la media, il che è auspicabile considerando che tutto il resto della riga è uguale e quindi si tratta molto probabilmente dello stesso oggetto, duplicato per errore, di cui dunque prendiamo un valore medio tra quelli presenti.

In [27]:
sdf = rdd.toDF()

sdf.createOrReplaceTempView('Bank_Loan_Dataset')

columns_temp = [col for col in columns if col != 'Credit_Score' and col != 'Annual_Income']
Project = ', '.join(columns_temp)

sql = """
SELECT {0}, BIGINT(AVG(Credit_Score)) AS Credit_Score, BIGINT(AVG(Annual_Income)) AS Annual_Income
FROM Bank_Loan_Dataset
GROUP BY {0}
""".format(Project)

sdf = spark.sql(sql)

del columns_temp

come si nota i Customer e Loan ID che si ripetevano nel dataset originale erano solo righe duplicate. Il dataset pulito non presenta nessuna riga uguale negli ID e possiamo quindi eliminarli. Anche escludendo questi attributi tutte le righe rimangono distinte.

In [28]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT COUNT(*) AS nbr_rows, COUNT(DISTINCT Customer_ID) AS nbr_customers, COUNT(DISTINCT Loan_ID) AS nbr_loans
FROM Bank_Loan_Dataset
"""

spark.sql(sql).show()

+--------+-------------+---------+
|nbr_rows|nbr_customers|nbr_loans|
+--------+-------------+---------+
|   74094|        74094|    74094|
+--------+-------------+---------+



In [29]:
columns = [col for col in columns if col != 'Customer_ID' and col != 'Loan_ID']

columns_categorical = [col.name for col in sdf.schema.fields if isinstance(col.dataType, StringType)]

columns_numerical = [col for col in columns if col not in columns_categorical]

sdf = sdf.select(columns)

In [30]:
sdf.printSchema()

root
 |-- Annual_Income: long (nullable = true)
 |-- Bankruptcies: long (nullable = true)
 |-- Credit_Score: long (nullable = true)
 |-- Current_Credit_Balance: long (nullable = true)
 |-- Current_Loan_Amount: long (nullable = true)
 |-- Home_Ownership: string (nullable = true)
 |-- Loan_Status: string (nullable = true)
 |-- Maximum_Open_Credit: long (nullable = true)
 |-- Monthly_Debt: double (nullable = true)
 |-- Months_since_last_delinquent: string (nullable = true)
 |-- Number_of_Credit_Problems: long (nullable = true)
 |-- Number_of_Open_Accounts: long (nullable = true)
 |-- Purpose: string (nullable = true)
 |-- Tax_Liens: long (nullable = true)
 |-- Term: string (nullable = true)
 |-- Years_in_current_job: double (nullable = true)
 |-- Years_of_Credit_History: double (nullable = true)



# Gestione dei Missing Values

In [32]:
nbr_nulls = get_nbr_nulls(spark_df=sdf, view_name = 'Bank_Loan_Dataset')

Annual_Income:      14947
Bankruptcies:        156
Credit_Score:      14947
Current_Credit_Balance:          0
Current_Loan_Amount:          0
Home_Ownership:          0
Loan_Status:          0
Maximum_Open_Credit:          2
Monthly_Debt:          0
Months_since_last_delinquent:          0
Number_of_Credit_Problems:          0
Number_of_Open_Accounts:          0
Purpose:          0
Tax_Liens:          7
Term:          0
Years_in_current_job:          0
Years_of_Credit_History:          0


Filliamo i Missing values degli attributi "Maximum_Open_Credit", "Bankruptcies" e "Tax_Liens" usando la Moda di diversi tipi di raggruppamenti.

per farlo usiamo la sintassi dell'SQL analitico, creando nuove apposite colonne.

In [32]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT *,
    BIGINT( PERCENTILE(Maximum_Open_Credit, 0.5) OVER(PARTITION BY Years_in_current_job,
                                                Home_Ownership,
                                                Number_of_Open_Accounts,
                                                Years_of_Credit_History) ) AS toFill_Maximum_Open_Credit,
    BIGINT( PERCENTILE(Bankruptcies, 0.5) OVER(PARTITION BY Months_since_last_delinquent,
                                                Number_of_Credit_Problems) ) AS toFill_Bankruptcies,
    BIGINT( PERCENTILE(Tax_Liens, 0.5) OVER(PARTITION BY Months_since_last_delinquent,
                                            Number_of_Credit_Problems) ) AS toFill_Tax_Liens
FROM Bank_Loan_Dataset
"""

sdf = spark.sql(sql)

In [33]:
def fill_nulls(row):
    d = row.asDict()
    if d['Maximum_Open_Credit'] is None:
        d['Maximum_Open_Credit'] = d['toFill_Maximum_Open_Credit']
    if d['Bankruptcies'] is None:
        d['Bankruptcies'] = d['toFill_Bankruptcies']
    if d['Tax_Liens'] is None:
        d['Tax_Liens'] = d['toFill_Tax_Liens']
    return Row(**d)

sdf = sdf.rdd.map(fill_nulls).toDF().select(columns)

In [34]:
nbr_nulls = get_nbr_nulls(spark_df=sdf, view_name = 'Bank_Loan_Dataset')

Annual_Income:      14947
Bankruptcies:          0
Credit_Score:      14947
Current_Credit_Balance:          0
Current_Loan_Amount:          0
Home_Ownership:          0
Loan_Status:          0
Maximum_Open_Credit:          0
Monthly_Debt:          0
Months_since_last_delinquent:          0
Number_of_Credit_Problems:          0
Number_of_Open_Accounts:          0
Purpose:          0
Tax_Liens:          0
Term:          0
Years_in_current_job:          0
Years_of_Credit_History:          0


Filliamo i Missing Values degli attributi "Credit_Score" e "Annual_Income" dividendo il dataset in clusters e usando la Media dei valori nei clusters.

Per effettuare il clustering dobbiamo considerare solo le colonne numeriche diverse da 'Credit_Score' e 'Annual_Income'.

In [35]:
columns_clustering = [col for col in columns_numerical if col != 'Credit_Score' and col != 'Annual_Income']

columns_clustering

['Bankruptcies',
 'Current_Credit_Balance',
 'Current_Loan_Amount',
 'Maximum_Open_Credit',
 'Monthly_Debt',
 'Number_of_Credit_Problems',
 'Number_of_Open_Accounts',
 'Tax_Liens',
 'Years_in_current_job',
 'Years_of_Credit_History']

In [36]:
X = sdf.select(columns_clustering).rdd.map(lambda row: Vectors.dense(row))

X_scaled = StandardScaler(withMean=True, withStd=True).fit(X).transform(X)

# Migliora modello

In [37]:
'''for k in range(1, 100):
    model = KMeans.train(X_scaled, k=10, maxIterations=20, initializationMode="random")'''

'for k in range(1, 100):\n    model = KMeans.train(X_scaled, k=10, maxIterations=20, initializationMode="random")'

In [38]:
model = KMeans.train(X_scaled, k=10, maxIterations=20, initializationMode="random")

In [39]:
def append_label(element):
    d = element[0].asDict()
    d['cluster_label'] = str(element[1])
    return Row(**d)

sdf = sdf.rdd.zip(model.predict(X_scaled)).map(append_label).toDF()

columns = sdf.schema.names

In [40]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT *,
    BIGINT( PERCENTILE(Credit_Score, 0.5) OVER(PARTITION BY cluster_label) ) AS toFill_Credit_Score,
    BIGINT( PERCENTILE(Annual_Income, 0.5) OVER(PARTITION BY cluster_label) ) AS toFill_Annual_Income
FROM Bank_Loan_Dataset
"""

sdf = spark.sql(sql)

In [41]:
sdf.printSchema()

root
 |-- Annual_Income: long (nullable = true)
 |-- Bankruptcies: long (nullable = true)
 |-- Credit_Score: long (nullable = true)
 |-- Current_Credit_Balance: long (nullable = true)
 |-- Current_Loan_Amount: long (nullable = true)
 |-- Home_Ownership: string (nullable = true)
 |-- Loan_Status: string (nullable = true)
 |-- Maximum_Open_Credit: long (nullable = true)
 |-- Monthly_Debt: double (nullable = true)
 |-- Months_since_last_delinquent: string (nullable = true)
 |-- Number_of_Credit_Problems: long (nullable = true)
 |-- Number_of_Open_Accounts: long (nullable = true)
 |-- Purpose: string (nullable = true)
 |-- Tax_Liens: long (nullable = true)
 |-- Term: string (nullable = true)
 |-- Years_in_current_job: double (nullable = true)
 |-- Years_of_Credit_History: double (nullable = true)
 |-- cluster_label: string (nullable = true)
 |-- toFill_Credit_Score: long (nullable = true)
 |-- toFill_Annual_Income: long (nullable = true)



In [44]:
sdf.show(1)

+-------------+------------+------------+----------------------+-------------------+--------------+-----------+-------------------+------------------+----------------------------+-------------------------+-----------------------+------------------+---------+----------+--------------------+-----------------------+-------------+-------------------+--------------------+
|Annual_Income|Bankruptcies|Credit_Score|Current_Credit_Balance|Current_Loan_Amount|Home_Ownership|Loan_Status|Maximum_Open_Credit|      Monthly_Debt|Months_since_last_delinquent|Number_of_Credit_Problems|Number_of_Open_Accounts|           Purpose|Tax_Liens|      Term|Years_in_current_job|Years_of_Credit_History|cluster_label|toFill_Credit_Score|toFill_Annual_Income|
+-------------+------------+------------+----------------------+-------------------+--------------+-----------+-------------------+------------------+----------------------------+-------------------------+-----------------------+------------------+---------+--

In [45]:
def fill_nulls(row):
    d = row.asDict()
    if d['Credit_Score'] is None:
        d['Credit_Score'] = d['toFill_Credit_Score']
    if d['Annual_Income'] is None:
        d['Annual_Income'] = d['toFill_Annual_Income']
    return Row(**d)

sdf = sdf.rdd.map(fill_nulls).toDF().select(columns)

In [46]:
sdf.printSchema()

root
 |-- Annual_Income: long (nullable = true)
 |-- Bankruptcies: long (nullable = true)
 |-- Credit_Score: long (nullable = true)
 |-- Current_Credit_Balance: long (nullable = true)
 |-- Current_Loan_Amount: long (nullable = true)
 |-- Home_Ownership: string (nullable = true)
 |-- Loan_Status: string (nullable = true)
 |-- Maximum_Open_Credit: long (nullable = true)
 |-- Monthly_Debt: double (nullable = true)
 |-- Months_since_last_delinquent: string (nullable = true)
 |-- Number_of_Credit_Problems: long (nullable = true)
 |-- Number_of_Open_Accounts: long (nullable = true)
 |-- Purpose: string (nullable = true)
 |-- Tax_Liens: long (nullable = true)
 |-- Term: string (nullable = true)
 |-- Years_in_current_job: double (nullable = true)
 |-- Years_of_Credit_History: double (nullable = true)
 |-- cluster_label: string (nullable = true)



In [47]:
nbr_nulls = get_nbr_nulls(spark_df=sdf, view_name = 'Bank_Loan_Dataset')

KeyboardInterrupt: 

In [ ]:
sdf.printSchema()

# ***

il tasso cedolare infatti coincide con il tasso governativo decennale Russo del 2016

In [46]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT COUNT(*) AS nbr_rows
FROM Bank_Loan_Dataset
"""

spark.sql(sql).show()

+--------+
|nbr_rows|
+--------+
|   69057|
+--------+



In [47]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT Loan_Status, COUNT(*) AS nbr_rows
FROM Bank_Loan_Dataset
GROUP BY Loan_Status """

spark.sql(sql).show()

+-----------+--------+
|Loan_Status|nbr_rows|
+-----------+--------+
| Fully Paid|   51088|
|Charged Off|   17969|
+-----------+--------+



In [48]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT Loan_Status, AVG(Credit_Score) AS avg_Credit_Score
FROM Bank_Loan_Dataset
GROUP BY Loan_Status """

spark.sql(sql).show()

+-----------+-----------------+
|Loan_Status| avg_Credit_Score|
+-----------+-----------------+
| Fully Paid|719.9645575699363|
|Charged Off|710.2000954350247|
+-----------+-----------------+



In [49]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT Loan_Status, AVG(Annual_Income) AS avg_Credit_Score
FROM Bank_Loan_Dataset
GROUP BY Loan_Status """

spark.sql(sql).show()

+-----------+------------------+
|Loan_Status|  avg_Credit_Score|
+-----------+------------------+
| Fully Paid|1410691.1453956058|
|Charged Off|1253940.9389215843|
+-----------+------------------+



In [ ]:
columns = sdf.schema.names

columns_categorical = [col.name for col in sdf.schema.fields if isinstance(col.dataType, StringType)]

columns_numerical = [col for col in columns if col not in columns_categorical]

In [42]:
sdf.write.parquet("hdfs://kddrtserver11.isti.cnr.it:9000/user/hpsa04/bank_loan_status_dataset")

In [50]:
sdf = spark.read.parquet("hdfs://kddrtserver11.isti.cnr.it:9000/user/hpsa04/bank_loan_status_dataset")

In [52]:
sdf.printSchema()

root
 |-- Loan_Status: string (nullable = true)
 |-- Current_Loan_Amount: long (nullable = true)
 |-- Term: string (nullable = true)
 |-- Credit_Score: double (nullable = true)
 |-- Annual_Income: double (nullable = true)
 |-- Years_in_current_job: string (nullable = true)
 |-- Home_Ownership: string (nullable = true)
 |-- Purpose: string (nullable = true)
 |-- Monthly_Debt: double (nullable = true)
 |-- Years_of_Credit_History: double (nullable = true)
 |-- Months_since_last_delinquent: string (nullable = true)
 |-- Number_of_Open_Accounts: string (nullable = true)
 |-- Number_of_Credit_Problems: string (nullable = true)
 |-- Current_Credit_Balance: long (nullable = true)
 |-- Maximum_Open_Credit: long (nullable = true)
 |-- Bankruptcies: string (nullable = true)
 |-- Tax_Liens: string (nullable = true)



# Ricordati!

anche se non esistono valori distinti  COUNT(\*)  può differire da COUNT(DISTINCT \*).

perché il primo conta tutte le righe mentre il secondo conta tutte e sole le righe dove non è presente neanche un NULL value

In [ ]:
sql = """
SELECT COUNT(*) AS nbr_rows
FROM Bank_Loan_Dataset
"""

spark.sql(sql).show()

In [ ]:
sql = """
SELECT COUNT(Credit_Score) AS nbr_rows
FROM Bank_Loan_Dataset
"""

spark.sql(sql).show()

In [ ]:
sql = """
SELECT COUNT(DISTINCT *) AS nbr_rows
FROM Bank_Loan_Dataset
"""

spark.sql(sql).show()

# tentativi

In [42]:
'''  tentativo non riuscito per fillare i missing values
rdd = sdf.rdd.map(lambda row: row.asDict())

rdd.keyBy(lambda row: (row['Years_in_current_job'],
                             row['Home_Ownership'],
                             row['Number_of_Open_Accounts'],
                             row['Years_of_Credit_History'])).groupByKey()
                             
def fill_null(d):
    if d['Maximum_Open_Credit'] == None:
        d['Maximum_Open_Credit'] = 
    return d

rdd.mapValues(fill_null)
'''

"  tentativo non riuscito\nrdd = sdf.rdd.map(lambda row: row.asDict())\n\nrdd.keyBy(lambda row: (row['Years_in_current_job'],\n                             row['Home_Ownership'],\n                             row['Number_of_Open_Accounts'],\n                             row['Years_of_Credit_History'])).groupByKey()\n                             \ndef fill_null(d):\n    if d['Maximum_Open_Credit'] == None:\n        d['Maximum_Open_Credit'] = \n    return d\n\nrdd.mapValues(fill_null)\n"